In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [5]:
X_train = pd.read_csv('D:\\DataScience\\Data\\data-science-london-scikit-learn\\train.csv')
df_train_lable=pd.read_csv('D:\\DataScience\\Data\\data-science-london-scikit-learn\\trainLabels.csv')
X_test = pd.read_csv('D:\\DataScience\\Data\\data-science-london-scikit-learn\\test.csv')

In [6]:
y = df_train_lable.values.ravel()
X_train_raw = X_train # Dữ liệu train gốc (40 cột)
X_test_raw = X_test   # Dữ liệu test gốc (40 cột)

# 2. Gộp dữ liệu để xử lý chung (QUAN TRỌNG)
X_all = np.r_[X_train_raw, X_test_raw]
print("Shape ban đầu:", X_all.shape)

# 3. Chuẩn hóa & Khử nhiễu bằng PCA (BƯỚC QUYẾT ĐỊNH ĐIỂM SỐ)
# PCA sẽ loại bỏ các cột rác, giữ lại phần tinh túy nhất
sc = StandardScaler()
X_all_scaled = sc.fit_transform(X_all)

pca = PCA(n_components=0.99, whiten=True, random_state=42)
X_all_pca = pca.fit_transform(X_all_scaled)
print("Shape sau PCA:", X_all_pca.shape) 
# (Nó sẽ giảm từ 40 cột xuống khoảng 12-15 cột quan trọng)

# 4. Tạo đặc trưng GMM từ dữ liệu đã sạch
# n_components=4 là con số "vàng" cho bộ dữ liệu này
gmm = GaussianMixture(n_components=4, covariance_type='full', random_state=42)
gmm.fit(X_all_pca)
gmm_features = gmm.predict_proba(X_all_pca)

# 5. Tách lại Train và Test
X_train_final = gmm_features[:len(y)]
X_test_final = gmm_features[len(y):]

# 6. Chia tập Train để Validation (Kiểm tra điểm)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_final, y, test_size=0.20, random_state=42
)

# 7. Chạy GridSearch với Random Forest
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'bootstrap': [True, False]
}

rfc = RandomForestClassifier(random_state=42)
grid = GridSearchCV(rfc, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit mô hình
grid.fit(X_train_split, y_train_split)

# 8. Xem kết quả
print("\n--- KẾT QUẢ ---")
print("Best Params:", grid.best_params_)
print("Điểm trên tập Validation:", grid.score(X_val_split, y_val_split))

Shape ban đầu: (9998, 40)
Shape sau PCA: (9998, 36)

--- KẾT QUẢ ---
Best Params: {'bootstrap': True, 'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Điểm trên tập Validation: 0.985
